In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.decomposition import PCA
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import game
from State import State

In [2]:
pd.options.display.max_columns = None
random.seed(10)
g = game.Game(quiet=True)

In [3]:
def act_and_append(iters=250000):
    g = game.Game(quiet=True)
    state = State(g)
    pca = [list(state.state)]
    for _ in range(iters):
        action = random.randrange(16)
        if ((len(g.dice) > 0) & (len([i for i in g.board if i.isnumeric()]) > 0)) | (g.jokers > 0):
            if g.board[action].isnumeric():
                next_die = g.dice[0]
                g.place_die(next_die,action+1)
                g.lock_n_roll()
            elif (g.jokers > 0) & (action not in g.joker_on_board):
                g.place_joker(action+1)
                g.lock_n_roll()
            elif len([i for i in g.board if i.isnumeric()]) > 0:
                next_die = g.dice[0]
                adjusted_pos = min([int(i) for i in g.board if i.isnumeric()],key = lambda x: abs(x-action))
                g.place_die(next_die,adjusted_pos)
                g.lock_n_roll()
            else:
                adjusted_pos = min([int(i) for i,v in enumerate(g.board) if i not in g.joker_on_board],key = lambda x: abs(x-action))
                g.place_joker(adjusted_pos+1)
        if g.gameover:
            g = game.Game(quiet=True)
        pca.append(list(State(g).state))
    return np.matrix(pca)

def normalize(data):
    df = data.copy()
    continuous = [c for c in df if len(df[c].unique()) > 2]
    for c in continuous: df[c] = (df[c] - df[c].mean()) / df[c].std()
    return df

In [4]:
matrix = act_and_append()
data = pd.DataFrame(columns=State(g).labels,data=matrix)

In [5]:
data.head()

,JokerAv,Die0_B,Die0_G,Die0_R,Die0_Y,Die0_1,Die0_2,Die0_3,Die0_4,Die1_B,Die1_G,Die1_R,Die1_Y,Die1_1,Die1_2,Die1_3,Die1_4,Die2_B,Die2_G,Die2_R,Die2_Y,Die2_1,Die2_2,Die2_3,Die2_4,Die3_B,Die3_G,Die3_R,Die3_Y,Die3_1,Die3_2,Die3_3,Die3_4,Space0_CoveredByC_R,Space0_CoveredByC_B,Space0_CoveredByC_G,Space0_CoveredByC_Y,Space0_CoveredByN_1,Space0_CoveredByN_2,Space0_CoveredByN_3,Space0_CoveredByN_4,Space0_CoveredByJO,Move_0_Dist,Space1_CoveredByC_R,Space1_CoveredByC_B,Space1_CoveredByC_G,Space1_CoveredByC_Y,Space1_CoveredByN_1,Space1_CoveredByN_2,Space1_CoveredByN_3,Space1_CoveredByN_4,Space1_CoveredByJO,Move_1_Dist,Space2_CoveredByC_R,Space2_CoveredByC_B,Space2_CoveredByC_G,Space2_CoveredByC_Y,Space2_CoveredByN_1,Space2_CoveredByN_2,Space2_CoveredByN_3,Space2_CoveredByN_4,Space2_CoveredByJO,Move_2_Dist,Space3_CoveredByC_R,Space3_CoveredByC_B,Space3_CoveredByC_G,Space3_CoveredByC_Y,Space3_CoveredByN_1,Space3_CoveredByN_2,Space3_CoveredByN_3,Space3_CoveredByN_4,Space3_CoveredByJO,Move_3_Dist,Space4_CoveredByC_R,Space4_CoveredByC_B,Space4_CoveredByC_G,Space4_CoveredByC_Y,Space4_CoveredByN_1,Space4_CoveredByN_2,Space4_CoveredByN_3,Space4_CoveredByN_4,Space4_CoveredByJO,Move_4_Dist,Space5_CoveredByC_R,Space5_CoveredByC_B,Space5_CoveredByC_G,Space5_CoveredByC_Y,Space5_CoveredByN_1,Space5_CoveredByN_2,Space5_CoveredByN_3,Space5_CoveredByN_4,Space5_CoveredByJO,Move_5_Dist,Space6_CoveredByC_R,Space6_CoveredByC_B,Space6_CoveredByC_G,Space6_CoveredByC_Y,Space6_CoveredByN_1,Space6_CoveredByN_2,Space6_CoveredByN_3,Space6_CoveredByN_4,Space6_CoveredByJO,Move_6_Dist,Space7_CoveredByC_R,Space7_CoveredByC_B,Space7_CoveredByC_G,Space7_CoveredByC_Y,Space7_CoveredByN_1,Space7_CoveredByN_2,Space7_CoveredByN_3,Space7_CoveredByN_4,Space7_CoveredByJO,Move_7_Dist,Space8_CoveredByC_R,Space8_CoveredByC_B,Space8_CoveredByC_G,Space8_CoveredByC_Y,Space8_CoveredByN_1,Space8_CoveredByN_2,Space8_CoveredByN_3,Space8_CoveredByN_4,Space8_CoveredByJO,Move_8_Dist,Space9_CoveredByC_R,Space9_CoveredByC_B,Space9_CoveredByC_G,Space9_CoveredByC_Y,Space9_CoveredByN_1,Space9_CoveredByN_2,Space9_CoveredByN_3,Space9_CoveredByN_4,Space9_CoveredByJO,Move_9_Dist,Space10_CoveredByC_R,Space10_CoveredByC_B,Space10_CoveredByC_G,Space10_CoveredByC_Y,Space10_CoveredByN_1,Space10_CoveredByN_2,Space10_CoveredByN_3,Space10_CoveredByN_4,Space10_CoveredByJO,Move_10_Dist,Space11_CoveredByC_R,Space11_CoveredByC_B,Space11_CoveredByC_G,Space11_CoveredByC_Y,Space11_CoveredByN_1,Space11_CoveredByN_2,Space11_CoveredByN_3,Space11_CoveredByN_4,Space11_CoveredByJO,Move_11_Dist,Space12_CoveredByC_R,Space12_CoveredByC_B,Space12_CoveredByC_G,Space12_CoveredByC_Y,Space12_CoveredByN_1,Space12_CoveredByN_2,Space12_CoveredByN_3,Space12_CoveredByN_4,Space12_CoveredByJO,Move_12_Dist,Space13_CoveredByC_R,Space13_CoveredByC_B,Space13_CoveredByC_G,Space13_CoveredByC_Y,Space13_CoveredByN_1,Space13_CoveredByN_2,Space13_CoveredByN_3,Space13_CoveredByN_4,Space13_CoveredByJO,Move_13_Dist,Space14_CoveredByC_R,Space14_CoveredByC_B,Space14_CoveredByC_G,Space14_CoveredByC_Y,Space14_CoveredByN_1,Space14_CoveredByN_2,Space14_CoveredByN_3,Space14_CoveredByN_4,Space14_CoveredByJO,Move_14_Dist,Space15_CoveredByC_R,Space15_CoveredByC_B,Space15_CoveredByC_G,Space15_CoveredByC_Y,Space15_CoveredByN_1,Space15_CoveredByN_2,Space15_CoveredByN_3,Space15_CoveredByN_4,Space15_CoveredByJO,Move_15_Dist
0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [6]:
data.describe()

,JokerAv,Die0_B,Die0_G,Die0_R,Die0_Y,Die0_1,Die0_2,Die0_3,Die0_4,Die1_B,Die1_G,Die1_R,Die1_Y,Die1_1,Die1_2,Die1_3,Die1_4,Die2_B,Die2_G,Die2_R,Die2_Y,Die2_1,Die2_2,Die2_3,Die2_4,Die3_B,Die3_G,Die3_R,Die3_Y,Die3_1,Die3_2,Die3_3,Die3_4,Space0_CoveredByC_R,Space0_CoveredByC_B,Space0_CoveredByC_G,Space0_CoveredByC_Y,Space0_CoveredByN_1,Space0_CoveredByN_2,Space0_CoveredByN_3,Space0_CoveredByN_4,Space0_CoveredByJO,Move_0_Dist,Space1_CoveredByC_R,Space1_CoveredByC_B,Space1_CoveredByC_G,Space1_CoveredByC_Y,Space1_CoveredByN_1,Space1_CoveredByN_2,Space1_CoveredByN_3,Space1_CoveredByN_4,Space1_CoveredByJO,Move_1_Dist,Space2_CoveredByC_R,Space2_CoveredByC_B,Space2_CoveredByC_G,Space2_CoveredByC_Y,Space2_CoveredByN_1,Space2_CoveredByN_2,Space2_CoveredByN_3,Space2_CoveredByN_4,Space2_CoveredByJO,Move_2_Dist,Space3_CoveredByC_R,Space3_CoveredByC_B,Space3_CoveredByC_G,Space3_CoveredByC_Y,Space3_CoveredByN_1,Space3_CoveredByN_2,Space3_CoveredByN_3,Space3_CoveredByN_4,Space3_CoveredByJO,Move_3_Dist,Space4_CoveredByC_R,Space4_CoveredByC_B,Space4_CoveredByC_G,Space4_CoveredByC_Y,Space4_CoveredByN_1,Space4_CoveredByN_2,Space4_CoveredByN_3,Space4_CoveredByN_4,Space4_CoveredByJO,Move_4_Dist,Space5_CoveredByC_R,Space5_CoveredByC_B,Space5_CoveredByC_G,Space5_CoveredByC_Y,Space5_CoveredByN_1,Space5_CoveredByN_2,Space5_CoveredByN_3,Space5_CoveredByN_4,Space5_CoveredByJO,Move_5_Dist,Space6_CoveredByC_R,Space6_CoveredByC_B,Space6_CoveredByC_G,Space6_CoveredByC_Y,Space6_CoveredByN_1,Space6_CoveredByN_2,Space6_CoveredByN_3,Space6_CoveredByN_4,Space6_CoveredByJO,Move_6_Dist,Space7_CoveredByC_R,Space7_CoveredByC_B,Space7_CoveredByC_G,Space7_CoveredByC_Y,Space7_CoveredByN_1,Space7_CoveredByN_2,Space7_CoveredByN_3,Space7_CoveredByN_4,Space7_CoveredByJO,Move_7_Dist,Space8_CoveredByC_R,Space8_CoveredByC_B,Space8_CoveredByC_G,Space8_CoveredByC_Y,Space8_CoveredByN_1,Space8_CoveredByN_2,Space8_CoveredByN_3,Space8_CoveredByN_4,Space8_CoveredByJO,Move_8_Dist,Space9_CoveredByC_R,Space9_CoveredByC_B,Space9_CoveredByC_G,Space9_CoveredByC_Y,Space9_CoveredByN_1,Space9_CoveredByN_2,Space9_CoveredByN_3,Space9_CoveredByN_4,Space9_CoveredByJO,Move_9_Dist,Space10_CoveredByC_R,Space10_CoveredByC_B,Space10_CoveredByC_G,Space10_CoveredByC_Y,Space10_CoveredByN_1,Space10_CoveredByN_2,Space10_CoveredByN_3,Space10_CoveredByN_4,Space10_CoveredByJO,Move_10_Dist,Space11_CoveredByC_R,Space11_CoveredByC_B,Space11_CoveredByC_G,Space11_CoveredByC_Y,Space11_CoveredByN_1,Space11_CoveredByN_2,Space11_CoveredByN_3,Space11_CoveredByN_4,Space11_CoveredByJO,Move_11_Dist,Space12_CoveredByC_R,Space12_CoveredByC_B,Space12_CoveredByC_G,Space12_CoveredByC_Y,Space12_CoveredByN_1,Space12_CoveredByN_2,Space12_CoveredByN_3,Space12_CoveredByN_4,Space12_CoveredByJO,Move_12_Dist,Space13_CoveredByC_R,Space13_CoveredByC_B,Space13_CoveredByC_G,Space13_CoveredByC_Y,Space13_CoveredByN_1,Space13_CoveredByN_2,Space13_CoveredByN_3,Space13_CoveredByN_4,Space13_CoveredByJO,Move_13_Dist,Space14_CoveredByC_R,Space14_CoveredByC_B,Space14_CoveredByC_G,Space14_CoveredByC_Y,Space14_CoveredByN_1,Space14_CoveredByN_2,Space14_CoveredByN_3,Space14_CoveredByN_4,Space14_CoveredByJO,Move_14_Dist,Space15_CoveredByC_R,Space15_CoveredByC_B,Space15_CoveredByC_G,Space15_CoveredByC_Y,Space15_CoveredByN_1,Space15_CoveredByN_2,Space15_CoveredByN_3,Space15_CoveredByN_4,Space15_CoveredByJO,Move_15_Dist
count,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,250001.000000,2

In [7]:
data_normed = normalize(data)
data_normed.head()

,JokerAv,Die0_B,Die0_G,Die0_R,Die0_Y,Die0_1,Die0_2,Die0_3,Die0_4,Die1_B,Die1_G,Die1_R,Die1_Y,Die1_1,Die1_2,Die1_3,Die1_4,Die2_B,Die2_G,Die2_R,Die2_Y,Die2_1,Die2_2,Die2_3,Die2_4,Die3_B,Die3_G,Die3_R,Die3_Y,Die3_1,Die3_2,Die3_3,Die3_4,Space0_CoveredByC_R,Space0_CoveredByC_B,Space0_CoveredByC_G,Space0_CoveredByC_Y,Space0_CoveredByN_1,Space0_CoveredByN_2,Space0_CoveredByN_3,Space0_CoveredByN_4,Space0_CoveredByJO,Move_0_Dist,Space1_CoveredByC_R,Space1_CoveredByC_B,Space1_CoveredByC_G,Space1_CoveredByC_Y,Space1_CoveredByN_1,Space1_CoveredByN_2,Space1_CoveredByN_3,Space1_CoveredByN_4,Space1_CoveredByJO,Move_1_Dist,Space2_CoveredByC_R,Space2_CoveredByC_B,Space2_CoveredByC_G,Space2_CoveredByC_Y,Space2_CoveredByN_1,Space2_CoveredByN_2,Space2_CoveredByN_3,Space2_CoveredByN_4,Space2_CoveredByJO,Move_2_Dist,Space3_CoveredByC_R,Space3_CoveredByC_B,Space3_CoveredByC_G,Space3_CoveredByC_Y,Space3_CoveredByN_1,Space3_CoveredByN_2,Space3_CoveredByN_3,Space3_CoveredByN_4,Space3_CoveredByJO,Move_3_Dist,Space4_CoveredByC_R,Space4_CoveredByC_B,Space4_CoveredByC_G,Space4_CoveredByC_Y,Space4_CoveredByN_1,Space4_CoveredByN_2,Space4_CoveredByN_3,Space4_CoveredByN_4,Space4_CoveredByJO,Move_4_Dist,Space5_CoveredByC_R,Space5_CoveredByC_B,Space5_CoveredByC_G,Space5_CoveredByC_Y,Space5_CoveredByN_1,Space5_CoveredByN_2,Space5_CoveredByN_3,Space5_CoveredByN_4,Space5_CoveredByJO,Move_5_Dist,Space6_CoveredByC_R,Space6_CoveredByC_B,Space6_CoveredByC_G,Space6_CoveredByC_Y,Space6_CoveredByN_1,Space6_CoveredByN_2,Space6_CoveredByN_3,Space6_CoveredByN_4,Space6_CoveredByJO,Move_6_Dist,Space7_CoveredByC_R,Space7_CoveredByC_B,Space7_CoveredByC_G,Space7_CoveredByC_Y,Space7_CoveredByN_1,Space7_CoveredByN_2,Space7_CoveredByN_3,Space7_CoveredByN_4,Space7_CoveredByJO,Move_7_Dist,Space8_CoveredByC_R,Space8_CoveredByC_B,Space8_CoveredByC_G,Space8_CoveredByC_Y,Space8_CoveredByN_1,Space8_CoveredByN_2,Space8_CoveredByN_3,Space8_CoveredByN_4,Space8_CoveredByJO,Move_8_Dist,Space9_CoveredByC_R,Space9_CoveredByC_B,Space9_CoveredByC_G,Space9_CoveredByC_Y,Space9_CoveredByN_1,Space9_CoveredByN_2,Space9_CoveredByN_3,Space9_CoveredByN_4,Space9_CoveredByJO,Move_9_Dist,Space10_CoveredByC_R,Space10_CoveredByC_B,Space10_CoveredByC_G,Space10_CoveredByC_Y,Space10_CoveredByN_1,Space10_CoveredByN_2,Space10_CoveredByN_3,Space10_CoveredByN_4,Space10_CoveredByJO,Move_10_Dist,Space11_CoveredByC_R,Space11_CoveredByC_B,Space11_CoveredByC_G,Space11_CoveredByC_Y,Space11_CoveredByN_1,Space11_CoveredByN_2,Space11_CoveredByN_3,Space11_CoveredByN_4,Space11_CoveredByJO,Move_11_Dist,Space12_CoveredByC_R,Space12_CoveredByC_B,Space12_CoveredByC_G,Space12_CoveredByC_Y,Space12_CoveredByN_1,Space12_CoveredByN_2,Space12_CoveredByN_3,Space12_CoveredByN_4,Space12_CoveredByJO,Move_12_Dist,Space13_CoveredByC_R,Space13_CoveredByC_B,Space13_CoveredByC_G,Space13_CoveredByC_Y,Space13_CoveredByN_1,Space13_CoveredByN_2,Space13_CoveredByN_3,Space13_CoveredByN_4,Space13_CoveredByJO,Move_13_Dist,Space14_CoveredByC_R,Space14_CoveredByC_B,Space14_CoveredByC_G,Space14_CoveredByC_Y,Space14_CoveredByN_1,Space14_CoveredByN_2,Space14_CoveredByN_3,Space14_CoveredByN_4,Space14_CoveredByJO,Move_14_Dist,Space15_CoveredByC_R,Space15_CoveredByC_B,Space15_CoveredByC_G,Space15_CoveredByC_Y,Space15_CoveredByN_1,Space15_CoveredByN_2,Space15_CoveredByN_3,Space15_CoveredByN_4,Space15_CoveredByJO,Move_15_Dist
0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.110433,0,0,0,0,0,0,0,0,0,-0.252842,0,0,0,0,0,0,0,0,0,-0.207212,0,0,0,0,0,0,0,0,0,-0.174791,0,0,0,0,0,0,0,0,0,-0.145517,0,0,0,0,0,0,0,0,0,-0.115898,0,0,0,0,0,0,0,0,0,-0.087716,0,0,0,0,0,0,0,0,0,-0.053669,0,0,0,0,0,0,0,0,0,-0.02389,0,0,0,0,0,0,0,0,0,-0.003111,0,0,0,0,0,0,0,0,0,0.019988,0,0,0,0,0,0,0,0,0,0.048845,0,0,0,0,0,0,0,0,0,0.076921,0,0,0,0,0,0,0,0,0,0.105706,0,0,0,0,0,0,0,0,0,0.136676,0,0,0,0,0,0,0,0,0,0.168066
1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.110433,0,1,0,0,1,0,0,0,0,-0.252842,0,0,0,0,0,0,0,0,0,-0

In [ ]:
pcs_names = [f'PC{i}' for i,v in enumerate(data.columns)]
pca = PCA(n_components=data.shape[1])
pca.fit(data_normed)
data_pca = pd.DataFrame(pca.transform(data_normed),columns = pcs_names)

In [ ]:
data_pca.head()

In [ ]:
# create a scree plot
sns.set(rc={'figure.figsize':(18,10)})
sns.lineplot(y = pca.explained_variance_ratio_, x = range(data_pca.shape[1]))
plt.ylabel('Explained variance')
plt.xlabel('Principal component')

In [ ]:
# select fewest number of components that will capture 90% of the total variance in the dataset
for pc, var in zip(pcs_names,np.cumsum(pca.explained_variance_ratio_)):
    print(pc,var)

In [ ]:
selected_cols = 16
data_reduced = data_pca.iloc[:,0:selected_cols]

In [ ]:
pca_transformation = PCA(n_components=selected_cols)
pca_transform.fit(pca)

In [ ]:
with open('model/pca.pckl','wb') as f:
    pickle.dump(pca_transform,f)